<a href="https://colab.research.google.com/github/nhat132940/CS114.L21.KHCL/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Thu thập headline từ 6 trang báo**
## **Lớp: CS114.L21.KHCL**
## **Thành viên nhóm:**
- Phan Minh Nhật - 19521956
- Lê Võ Tiến Phát - 19521993
- Hồ Thịnh - 19522274

**Các trang báo châm biếm:**
- The Daily Squib - UK: https://www.dailysquib.co.uk/
- The Betoota Advocate - Australia : https://www.betootaadvocate.com/
- The Onion - US : https://www.theonion.com/

**Các trang báo chính thống:**
- NBC News - US : https://www.nbcnews.com/
- Yahoo News - UK : https://news.yahoo.com/
- Daily Telegraph - Australia: https://www.dailytelegraph.com.au/

References:
- Code này bọn em tham khảo của một nhóm khác. Link: https://github.com/caohungphu/CS114.L21/find/main
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/


In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime
import time
import pandas as pd
import json


**Code để crawl trang The Onion**

In [ ]:
def is_leap_year(year):
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)    
  
def getDate(month, year):
    day_of_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    if is_leap_year(year):
        day_of_month[2] = 29
    return day_of_month[month]

def getNameMonth(month):
    if month < 1 or month > 12:
        return 'january'
    name_of_month = ['', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    return name_of_month[month]
    
def getFullDate(day, month, year):
    return "{}/{}/{}".format(day, month, year)

def getArticleFromPage(day, month, year):
    urlPage = 'https://www.theonion.com/sitemap/{}/{}/{}'.format(str(year), getNameMonth(month), str(day))
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
    response = requests.get(urlPage, headers = headers)
    soupSite = BeautifulSoup(response.text, 'html.parser')
    soupArticle = soupSite.find_all("h4", class_="sc-1w8kdgf-1 bwRmiu js_sitemap-article")
    return soupArticle

def getHeadlineDataFormArticle(soupArticle):
    result = []
    for x in soupArticle:
        x = x.find("a")
        headline = x.get_text()
        article_link = x['href']
        is_sarcastic = '1'
        result.append([ is_sarcastic, headline, article_link])
    return result

def writeFile(headlineData):
    fileOutput = 'theonion.json'
    f = open(fileOutput, "a", encoding="utf-8")
    for i in headlineData:
        strOut = "is_sarcastic:" + i[0] + ", headline: " + i[1] + ", article_link: " + i[2] + "\n"
        f.writelines(strOut)
    f.close()
        
def getDataFromPage(day, month, year):
    print("Date: {}/{}/{}:".format(day, month, year))
    soupArticle = getArticleFromPage(day, month, year)
    headlineData = getHeadlineDataFormArticle(soupArticle)
    writeFile(headlineData)
    print("Number of data: ",len(headlineData))
    print("\n")

if __name__ == "__main__":
    year = 2021
    year_stop = 2020
    while year > year_stop - 1:
        month = 12
        if year == 2021:
            month = 6
        while month:
            day = getDate(month, year)
            if year == 2021 and month == 6:
                day = 7
            while day: 
                getDataFromPage(day, month, year)
                day -= 1
            month -= 1
        year -= 1


**Code để crawl trang The Betoota Advocate**

In [ ]:
def getArticleFromPage(page):
    API_url = "https://www.betootaadvocate.com/category/breaking-news/page/{}/".format(str(page))
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
    response = requests.get(API_url, headers = headers)
    soupSite = BeautifulSoup(response.text, 'html.parser')
    soupArticle = soupSite.find_all("h3", class_="entry-title td-module-title")
    print(response)
    print(soupArticle)
    return soupArticle

def getHeadlineDataFormArticle(soupArticle):
    result = []
    for x in soupArticle:
        headline = x.find("a")['title']
        article_link = x.find("a")['href']
        is_sarcastic = '1'
        result.append([is_sarcastic, headline, article_link])
    return result

def writeFile(headlineData):
    fileOutput = 'betootaadvocate.json'
    f = open(fileOutput, "a", encoding="utf-8")
    for i in headlineData:
        strOut = "is_sarcastic:" + i[0] + ", headline: " + i[1] + ", article_link: " + i[2] + "\n"
        f.writelines(strOut)
    f.close()
        
def getDataFromPage(page):
    print(" Page: {}".format(page)) 
    soupArticle = getArticleFromPage(page)
    headlineData = getHeadlineDataFormArticle(soupArticle)
    writeFile(headlineData)
    print("Number of data: ",len(headlineData))
    print("\n")

if __name__ == "__main__":
    page = 1
    while page < 86: #6 January 2018
        getDataFromPage(page)
        page += 1
   

**Code để crawl trang The Daily Squib**

In [ ]:
def getArticleFromPage(page):
    API_url = "https://www.dailysquib.co.uk/category/world/page/{}/".format(str(page))
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
    response = requests.get(API_url, headers = headers)
    soupSite = BeautifulSoup(response.text, 'html.parser')
    soupArticle = soupSite.find_all("div", class_="td-module-thumb")
    print(response)
    print(soupArticle)
    return soupArticle

def getHeadlineDataFormArticle(soupArticle):
    result = []
    for x in soupArticle:
        headline = x.find("a")['title']
        article_link = x.find("a")['href']
        is_sarcastic = '1'
        result.append([is_sarcastic, headline, article_link])
    return result

def writeFile(headlineData):
    fileOutput = 'dailysquib.json'
    f = open(fileOutput, "a", encoding="utf-8")
    for i in headlineData:
        strOut = "is_sarcastic:" + i[0] + ", headline: " + i[1] + ", article_link: " + i[2] + "\n"
        f.writelines(strOut)
    f.close()
        
def getDataFromPage(page):
    print(" Page: {}".format(page)) 
    soupArticle = getArticleFromPage(page)
    headlineData = getHeadlineDataFormArticle(soupArticle)
    writeFile(headlineData)
    print("Number of data: ",len(headlineData))
    print("\n")

if __name__ == "__main__":
    page = 1
    while page < 108: #January 13, 2018
        getDataFromPage(page)
        page += 1
   

**Code để crawl trang NBC News**

In [ ]:
import time

def getNameMonth(month):
    if month < 1 or month > 12:
        return 'january'
    name_of_month = ['', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    return name_of_month[month]

def getArticleFromPage(month, year):
    API_url = 'https://www.nbcnews.com/archive/articles/{}/{}'.format(str(year), getNameMonth(month))
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
    response = requests.get(API_url, headers = headers)
    soupSite = BeautifulSoup(response.text, 'html.parser')
    soupArticle = soupSite.find_all("main", class_="MonthPage")
    print(response)
    return soupArticle

def getHeadlineDataFormArticle(soupArticle):
    result = []
    for x in soupArticle:
        x = x.find("a")
        headline = x.get_text()
        article_link = x['href']
        is_sarcastic = '0'
        result.append([ is_sarcastic, headline, article_link])
    return result

def writeFile(headlineData):
    fileOutput = 'nbcnews.json'
    f = open(fileOutput, "a", encoding="utf-8")
    for i in headlineData:
        strOut = "is_sarcastic:" + i[0] + ", headline: " + i[1] + ", article_link: " + i[2] + "\n"
        f.writelines(strOut)
    f.close()

def getDataFromPage(month, year):
    print("{}/{}:".format(month, year)) 
    soupArticle = getArticleFromPage(month, year)
    headlineData = getHeadlineDataFormArticle(soupArticle)
    writeFile(headlineData)
    print("Number of data: ",len(headlineData))
    print("\n")

if __name__ == "__main__":
    year = 2021
    year_stop = 2018
    while year > year_stop - 1:
        if year == 2021:
            month = 6
        else:
            month = 12
        while month:
            getDataFromPage(month, year)
            month -= 1
        year -= 1


**Code để crawl trang Yahoo News**













In [ ]:
def is_leap_year(year):
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)    
  
def getDate(month, year):
    day_of_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    if is_leap_year(year):
        day_of_month[2] = 29
    return day_of_month[month]

def getFullDate(day, month, year):
    return "{}/{}/{}".format(day, month, year)

def getArticleFromPage(day, month, year):
    urlPage = 'https://news.yahoo.com/sitemap/{}_{}_{}'.format(str(year), str(month), str(day))
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
    response = requests.get(urlPage, headers = headers)
    soupSite = BeautifulSoup(response.text, 'html.parser')
    soupArticle = soupSite.find_all("a", class_="Td(n) Td(u):h C($c-fuji-grey-k) Fz(16px) Ta(start) Mb(20px) D(b) Cur(p)")
    print(response)
    return soupArticle

def getHeadlineDataFormArticle(soupArticle):
    result = []
    for x in soupArticle:
        headline = x.get_text()
        article_link = x['href']
        is_sarcastic = '0'
        result.append([ is_sarcastic, headline, article_link])
    return result

def writeFile(headlineData):
    fileOutput = 'yahoo.json'
    f = open(fileOutput, "a", encoding="utf-8")
    for i in headlineData:
        strOut = "is_sarcastic:" + i[0] + ", headline: " + i[1] + ", article_link: " + i[2] + "\n"
        f.writelines(strOut)
    f.close()
        
def getDataFromPage(day, month, year):
    print("Date: {}_{}_{}:".format(day, month, year))
    soupArticle = getArticleFromPage(day, month, year)
    headlineData = getHeadlineDataFormArticle(soupArticle)
    writeFile(headlineData)
    print("Number of data: ",len(headlineData))
    print("\n")

if __name__ == "__main__":
    year = 2021
    year_stop = 2018
    while year > year_stop - 1:
        month = 12
        if year == 2021:
            month = 6
        while month:
            day = getDate(month, year)
            if year == 2021 and month == 6:
                day = 7
            while day: 
                getDataFromPage(day, month, year)
                day -= 1
            month -= 1
        year -= 1

**Code để crawl trang Daily Telegraph**

In [ ]:
def getArticleFromPage(page):
    API_url = "https://www.dailytelegraph.com.au/news/breaking-news/page/{}/".format(str(page))
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
    response = requests.get(API_url, headers = headers)
    soupSite = BeautifulSoup(response.text, 'html.parser')
    soupArticle = soupSite.find_all("a", class_="storyblock_title_link")
    print(response)
    print(soupArticle)
    return soupArticle

def getHeadlineDataFormArticle(soupArticle):
    result = []
    for x in soupArticle:
        headline = x.get_text()
        article_link = x['href']
        is_sarcastic = '0'
        result.append([is_sarcastic, headline, article_link])
    return result

def writeFile(headlineData):
    fileOutput = 'dailytelegraph.json'
    f = open(fileOutput, "a", encoding="utf-8")
    for i in headlineData:
        strOut = "is_sarcastic:" + i[0] + ", headline: " + i[1] + ", article_link: " + i[2] + "\n"
        f.writelines(strOut)
    f.close()
        
def getDataFromPage(page):
    print(" Page: {}".format(page)) 
    soupArticle = getArticleFromPage(page)
    headlineData = getHeadlineDataFormArticle(soupArticle)
    writeFile(headlineData)
    print("Number of data: ",len(headlineData))
    print("\n")

if __name__ == "__main__":
    page = 1
    while page < 200:
        getDataFromPage(page)
        page += 1
   

## **Báo cáo:**
- Bọn em đã thu thập được 64411 headline từ 6 trang báo.
- Trong đó các headline crawl được từ The Daily Squib và The Betoota Advocate có các headline bị trùng.
- Headline thu thập từ trang Yahoo News có các headline chữ Trung Quốc và các headline bị lỗi (có nhiều dấu ?).
Do số lượng headline khá nhiều nên bọn em sẽ xử lí các vấn đề trên sau.
Ước tính sau khi xử lí sẽ mất khoảng 6000-10000 headline.
 